<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/%EC%9D%B4%ED%83%88%EC%98%88%EC%B8%A1(%EB%B6%84%EB%A5%98)_4%EB%AA%A8%EB%8D%B8(%EC%9D%98%EC%82%AC%EA%B2%B0%EC%A0%95%2C%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%2C%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8%2C%EC%84%9C%ED%8F%AC%ED%8A%B8%EB%B2%A1%ED%84%B0%EB%A8%B8%EC%8B%A0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#https://www.datamanim.com/dataset/03_dataq/typetwo.html
#분류
#서비스 이탈예측 데이터
#데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
x_train <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv",header=T, stringsAsFactors=T)
y_train <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv",header=T, stringsAsFactors=T)
x_test <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv",header=T, stringsAsFactors=T)
x_label <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv",header=T, stringsAsFactors=T)
#install.packages("caret")
#install.packages("ModelMetrics")
#install.packages("randomForest")
#install.packages("rpart")
#install.packages("car")
#install.packages("lmtest")
#install.packages("ROCR")
library(dplyr)
library(caret)
library(car)
library(lmtest)
library(ROCR)
library(ModelMetrics)
library(randomForest)
library(rpart)
library(e1071)
# 결측치 확인
#apply(is.na(x_train),2,sum)
#apply(is.na(x_test),2,sum)
#apply(is.na(y_train),2,sum)

x_train<-merge(x_train,y_train,by='CustomerId')
x_test<-merge(x_test,x_label,by='CustomerId')


x_test<-x_test[,c(3,6,7,8,9,10,11,12,13)]
x_train<-x_train[,c(3,6,7,8,9,10,11,12,13)]

m_test<-x_test
m_test$Exited<-as.factor(m_test$Exited)
m_train<-x_train
m_train$Exited<-as.factor(m_train$Exited)



str(m_test)
m_test$Exited

m1 <- rpart(Exited~., data=x_train) #의사결정나무
m2 <- glm(Exited~., data=x_train, family="binomial") #로지스틱 회귀분석
m3 <- randomForest(Exited~., data=m_train, ntree=100) #랜덤포레스트
m4 <- svm(Exited~., data=x_train)#서포트벡터머신

pd1<-predict(m1,newdata=x_test)
pd2<-predict(m2,newdata=x_test)
pd3<-predict(m3,newdata=m_test, type = "response")
pd4<-predict(m4,newdata=x_test)

prd1 <- prediction(pd1, x_test$Exited)
prd2 <- prediction(pd2, x_test$Exited)
prd3 <- prediction(pd3, m_test$Exited)
prd4 <- prediction(pd4, x_test$Exited)
pf1<-performance(prd1, measure = "auc")
pf2<-performance(prd2, measure = "auc")
pf3<-performance(prd3, measure = "auc")
pf4<-performance(prd4, measure = "auc")
cat("ctree auc:",pf1@y.values[[1]],'\n')
cat("glm auc:",pf2@y.values[[1]],'\n')
cat("randomForest auc:",pf3@y.values[[1]],'\n')
cat("svm auc:",pf4@y.values[[1]],'\n')

'data.frame':	3501 obs. of  9 variables:
 $ CreditScore    : int  601 745 845 618 787 586 683 724 573 777 ...
 $ Age            : int  47 48 28 37 25 33 32 33 44 30 ...
 $ Tenure         : int  1 10 9 5 5 10 0 4 9 4 ...
 $ Balance        : num  64430 96049 0 96653 0 ...
 $ NumOfProducts  : int  2 1 2 1 2 2 2 1 2 1 ...
 $ HasCrCard      : int  0 1 1 1 1 1 1 0 1 1 ...
 $ IsActiveMember : int  1 0 1 0 0 1 1 1 0 0 ...
 $ EstimatedSalary: num  96518 74511 56186 98686 47308 ...
 $ Exited         : Factor w/ 2 levels "0","1": 1 1 1 2 1 1 1 2 1 2 ...


[1] 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1
  [38] 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0
  [75] 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1
 [112] 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 [149] 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0
 [186] 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
 [223] 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 [260] 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0
 [297] 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0
 [334] 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0
 [371] 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0
 [408] 0 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0
 [445] 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1
 [482] 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0
 [519] 0 0 1 0 1 1 0 0 0 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0
 [556] 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0
 [593] 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0
 [630] 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 1
 [667] 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
 [704] 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 [741] 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 [778] 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0
 [815] 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0
 [852] 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 [889] 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
 [926] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0
 [963] 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0
[1000] 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
[1037] 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1
[1074] 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
[1111] 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0
[1148] 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
[1185] 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0
[1222] 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 0
[1259] 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0
[1296] 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0
[1333] 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1
[1370] 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0
[1407] 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0
[1444] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0
[1481] 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
[1518] 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0
[1555] 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0
[1592] 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0
[1629] 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
[1666] 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
[1703] 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
[1740] 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
[1777] 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
[1814] 1 0 0 0 0 0 0 1 0 1 0 1 0 0

ERROR: ignored

In [1]:
#https://www.datamanim.com/dataset/03_dataq/typetwo.html
#분류
#서비스 이탈예측 데이터
#데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)


install.packages("caret")
install.packages("ModelMetrics")
install.packages("randomForest")
install.packages("rpart")
install.packages("car")
install.packages("lmtest")
install.packages("ROCR")
install.packages("dplyr")
library(dplyr)
library(caret)
library(car)
library(lmtest)
library(ROCR)
library(ModelMetrics)
library(randomForest)
library(rpart)
library(e1071)

dfx_train <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv",header=T, stringsAsFactors=T)
dfy_train <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv",header=T, stringsAsFactors=T)
dfx_test <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv",header=T, stringsAsFactors=T)
dfy_test <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv",header=T, stringsAsFactors=T)

# 결측치 확인
#apply(is.na(x_train),2,sum)
#apply(is.na(x_test),2,sum)
#apply(is.na(y_train),2,sum)

df<-merge(dfx_train,dfy_train,by='CustomerId')
str(df)
df<-df[,c(3,6,7,8,9,10,11,12,13)]

head(dfx_test)
df2_test<-dfx_test[,c(3,6,7,8,9,10,11,12)]
dfy_test<-dfy_test[,2]
# dfy_test<-as.factor(dfy_test)


idx<-sample(1:nrow(df), 0.7*nrow(df))
#idx <- createDataPartition(df$Exited, p=0.7, list=F)
df_train<-df[idx,]
df_test<-df[-idx,]

df_train$Exited<-as.factor(df_train$Exited)
df_test$Exited<-as.factor(df_test$Exited)

m1 <- rpart(Exited~., data=df_train) #의사결정나무
m2 <- glm(Exited~., data=df_train, family="binomial") #로지스틱 회귀분석
m3 <- randomForest(Exited~., data=df_train, ntree=100) #랜덤포레스트
m4 <- svm(Exited~., data=df_train)#서포트벡터머신

pd1<-predict(m1,newdata=df_test)
pd2<-predict(m2,newdata=df_test, type = "response")
pd3<-predict(m3,newdata=df_test)
pd4<-predict(m4,newdata=df_test)

p11<-as.factor(round(pd1[,2]))
p22<-as.factor(round(pd2))
p33<-pd3
p44<-pd4

caret::confusionMatrix(df_test$Exited, p11)$byClass[7]
caret::confusionMatrix(df_test$Exited, p22)$byClass[7]
caret::confusionMatrix(df_test$Exited, p33)$byClass[7]
caret::confusionMatrix(df_test$Exited, p44)$byClass[7]

caret::confusionMatrix(df_test$Exited, p11)$overall[1]
caret::confusionMatrix(df_test$Exited, p22)$overall[1]
caret::confusionMatrix(df_test$Exited, p33)$overall[1]
caret::confusionMatrix(df_test$Exited, p44)$overall[1]

auc(df_test$Exited, p11) #의사결정나무
auc(df_test$Exited, p22) #로지스틱 회귀분석
auc(df_test$Exited, p33) #랜덤포레스트
auc(df_test$Exited, p44) #서포트벡터머신


df$Exited <- as.factor(df$Exited)
m <- randomForest(Exited~., data=df, ntree=100)
#test 예측하기
p <- predict(m, newdata=df2_test)
head(p)
length(p)
result<-as.data.frame(as.character(p));head(result)
write.csv(result, "1234.csv", row.names=F)
abc<-read.csv("1234.csv")

tmp_f<-caret::confusionMatrix(as.factor(dfy_test), as.factor(p))
auc<- auc(dfy_test, as.numeric(p))
cat("auc=",auc)
tmp_f$byClass[7]
tmp_f$overall[1]
# 답안 제출 참고
# 아래 코드 변수명과 수험번호를 개인별로 변경하여 활용
# write.csv(변수명,'003000000.csv',row.names=F) 


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘zoo’


Installing package in

'data.frame':	6499 obs. of  13 variables:
 $ CustomerId     : int  15565701 15565706 15565779 15565806 15565878 15565891 15565996 15566030 15566091 15566111 ...
 $ Surname        : Factor w/ 2289 levels "Abazu","Abbott",..: 680 27 1087 2044 142 560 80 2076 2022 641 ...
 $ CreditScore    : int  698 612 627 532 631 709 653 497 545 596 ...
 $ Geography      : Factor w/ 3 levels "France","Germany",..: 3 3 2 1 3 1 1 2 3 1 ...
 $ Gender         : Factor w/ 4 levels " male","female",..: 3 4 3 1 4 4 4 4 3 4 ...
 $ Age            : int  39 35 30 38 29 39 44 41 32 39 ...
 $ Tenure         : int  9 1 6 9 3 8 8 5 4 9 ...
 $ Balance        : num  161994 0 57809 0 0 ...
 $ NumOfProducts  : int  1 1 1 2 2 2 2 1 1 1 ...
 $ HasCrCard      : int  0 1 1 0 1 1 1 0 1 1 ...
 $ IsActiveMember : int  0 1 0 0 1 0 1 0 0 0 ...
 $ EstimatedSalary: num  90212 83256 188258 30584 197963 ...
 $ Exited         : int  0 1 0 0 0 0 0 1 0 0 ...


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
,<int>,<fct>,<int>,<fct>,<fct>,<int>,<int>,<dbl>,<int>,<int>,<int>,<dbl>
1,15601012,Abdullah,802,France,Female,60,3,92887.06,1,1,0,39473.63
2,15734762,Ignatiev,602,France,Female,56,3,115895.22,3,1,0,4176.17
3,15586757,Anenechukwu,801,France,Female,32,4,75170.54,1,1,1,37898.50
4,15590888,Wade,693,Spain,Female,34,10,107556.06,2,0,0,154631.35
5,15726087,Ch'in,592,France,Female,62,5,0.00,1,1,1,100941.57
6,15769504,Munro,743,Germany,Female,34,1,131736.88,1,1,1,108543.21


F1 
0.9147193

F1 
0.892154

...

[1] 3501

,as.character(p)
,<chr>
1,1
2,1
3,0
4,0
5,0
6,0


auc= 0.683441

F1 
0.9149402

Accuracy 
0.8557555